# Session 2

# Randomized Benchmarking


## Introduction

**Randomization benchmarking (RB)** is a technique to estimate average gate performance, by leveraging self-inverting circuits (i.e., circuits whose composite action is to, ideally, implement the identity gate.)

Qiskit Ignis has tools to generate one- and two-qubit RB sequences based on self-inverting circuits comprised of Clifford gates.

This notebook gives an example for how to use the ``ignis.verification.randomized_benchmarking`` module. This particular example shows how to run 2-qubit randomized benchmarking (RB) simultaneous with 1-qubit RB. There are also examples on how to use some of the companion functions for predicting RB fidelity.

For more details on RB, see [this chapter](https://qiskit.org/textbook/ch-quantum-hardware/randomized-benchmarking.html) in the Qiskit textbook.

In [ ]:
#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Import Qiskit classes
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()

In [ ]:
hub = 'ibm-q'
group = 'open'
project = 'main'

provider = IBMQ.get_provider(hub, group, project)

provider.backends()

In [ ]:
backend = provider.get_backend('ibmq_valencia')
props = backend.properties()

Let's take a look at some of the properties of the gates on Valencia.

For details on these gates, see this documentation on [Qiskit's circuit library](https://qiskit.org/documentation/apidoc/circuit_library.html).

In [ ]:
# General single-qubit unitary
props.gate_property('u3')

In [ ]:
# Average single-qubit gate error across all 5 qubits
np.round(np.mean(np.array([props.gate_property('u3')[(j,)]['gate_error'][0] for j in range(5)])), 5)

In [ ]:
# Two-qubit CNOT gate
props.gate_property('cx')

## Generate a 1-qubit RB experiment design

An RB experiment design is a set of circuits that act on some number of qubits. Each circuit has the form $C_{1}C_{2}\cdots C_{L}C^{-1}$, where $C_{j}$ are Clifford gates, and $C = C_{1}C_{2}\cdots C_{L}$. In the ideal case, this circuit is self-inverting: $C_{1}C_{2}\cdots C_{L}C^{-1} = I$.

Given a length $L$, there are many Clifford circuits with that length. Consequently, we need to sample over the Clifford group to generate an ensemble of circuits with that length.

First, we'll start with a 1-qubit RB experiment design.

In [ ]:
#Number of Cliffords in the sequence (start, stop, steps)
lengths = np.arange(1,50,2)

#Number of seeds (random sequences) for each length
nSeeds = 10

# Specify that we want to do RB on qubit 0 on the backend
rb_pattern = [[0]]

Now that we've set the parameters of the expeirment design, we need to generate the circuits and some other data necessary to analyze the results.

We use Qiskit's`rb.randomized_benchmarking_seq` to do this.

This function returns:

- **rb_circs:** A list of lists of circuits for the rb sequences (separate list for each seed).
- **xdata:** The Clifford lengths (with multiplier if applicable).

In [ ]:
rb_circs, xdata = rb.randomized_benchmarking_seq(length_vector=lengths, nseeds=nSeeds, rb_pattern=rb_pattern)

The `rb_circs` are indexed as `rb_circs[seed][length]`. Let's look at some.

In [ ]:
# First seed, first length
rb_circs[0][0].draw(output='mpl')

In [ ]:
# First seed, last length
rb_circs[0][-1].draw(output='mpl')

## Simulate running RB

Here, we'll simulate running RB on hardware. 

We define a noise model for the simulator. Since we're doing single-qubit RB, we'll make a simple noise model of  depolarizing error probabilities to the  $u$-gates.

In [ ]:
noise_model = NoiseModel()
p1Q = 0.00079

noise_model.add_all_qubit_quantum_error(depolarizing_error(p1Q, 1), 'u2')
noise_model.add_all_qubit_quantum_error(depolarizing_error(2*p1Q, 1), 'u3')

## Execute the RB sequences

We simulate the RB sequences either using Qiskit simulator (with the noise model above).

In [ ]:
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
result_list = []
transpile_list = []
import time
for rb_seed,rb_circ_seed in enumerate(rb_circs):
    print('Compiling seed %d'%rb_seed)
    rb_circ_transpile = qiskit.transpile(rb_circ_seed, basis_gates=basis_gates)
    print('Simulating seed %d'%rb_seed)
    job = qiskit.execute(rb_circ_transpile, noise_model=noise_model, shots=shots, backend=backend,\
                         backend_options={'max_parallel_experiments': 0})
    result_list.append(job.result())
    transpile_list.append(rb_circ_transpile)    
print("Finished Simulating")

## Fit the RB results to a model

### Get statistics about the survival probabilities

The results in **result_list** should fit to an exponentially decaying function $A \cdot \alpha ^ m + B$, where $m$ is the Clifford length.

From $\alpha$ we can calculate the **Error per Clifford (EPC)**:
$$ \mathrm{EPC} = \frac{2^n-1}{2^n} (1-\alpha)$$
(where $n$ is the number of qubits).

In [ ]:
#Create an RBFitter object and fit the data
rbfit = rb.fitters.RBFitter(result_list, xdata)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
rbfit.plot_rb_data(ax=ax)

## Generate a 2-qubit RB experiment design

As in the 1-qubit case, an RB experiment design is a set of circuits that act on some number of qubits. Each circuit has the form $C_{1}C_{2}\cdots C_{L}C^{-1}$, where $C_{j}$ are Clifford gates, and $C = C_{1}C_{2}\cdots C_{L}$. In the ideal case, this circuit is self-inverting: $C_{1}C_{2}\cdots C_{L}C^{-1} = I$.

For the 2-qubit RB experiment design, each Clifford will act on 2 qubits.

Given a length $L$, there are many Clifford circuits with that length. Consequently, we need to sample over the Clifford group to generate an ensemble of circuits with that length.

In [ ]:
#Number of Cliffords in the sequence (start, stop, steps)
lengths = np.arange(1,50,2)

#Number of seeds (random sequences) for each length
nSeeds = 5

# Specify that we want to do RB on qubits 0 and 1 on the backend
rb_pattern = [[0, 1]]

In [ ]:
rb_circs, xdata = rb.randomized_benchmarking_seq(length_vector=lengths, nseeds=nSeeds, rb_pattern=rb_pattern)

In [ ]:
# First seed, first length
rb_circs[0][0].draw(output='mpl')

In [ ]:
# First seed, last length
rb_circs[0][-1].draw(output='mpl')

## Simulate running RB
Here, we'll simulate running RB on hardware. 

We define a noise model for the simulator. Since we're doing two-qubit RB, we'll need to include noise that acts on the 2-qubit CNOT gate, in addition to noise acting on the single-qubit $u$ gates.

In [ ]:
noise_model = NoiseModel()
p1Q = 0.00079
p2Q = .01

# For the single-qubit gates ,we'll use the same noise model as before...
noise_model.add_all_qubit_quantum_error(depolarizing_error(p1Q, 1), 'u2')
noise_model.add_all_qubit_quantum_error(depolarizing_error(2*p1Q, 1), 'u3')

# ...but add in noise on the CNOT gate
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2Q, 2), 'cx')

## Execute the RB sequences

We simulate the RB sequences either using Qiskit simulator (with the noise model above).

In [ ]:
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
shots = 200
result_list = []
transpile_list = []
import time
for rb_seed,rb_circ_seed in enumerate(rb_circs):
    print('Compiling seed %d'%rb_seed)
    rb_circ_transpile = qiskit.transpile(rb_circ_seed, basis_gates=basis_gates)
    print('Simulating seed %d'%rb_seed)
    job = qiskit.execute(rb_circ_transpile, noise_model=noise_model, shots=shots, backend=backend,\
                         backend_options={'max_parallel_experiments': 0})
    result_list.append(job.result())
    transpile_list.append(rb_circ_transpile)    
print("Finished Simulating")

## Fit the RB results to a model

### Get statistics about the survival probabilities

The results in **result_list** should fit to an exponentially decaying function $A \cdot \alpha ^ m + B$, where $m$ is the Clifford length.

From $\alpha$ we can calculate the **Error per Clifford (EPC)**:
$$ \mathrm{EPC} = \frac{2^n-1}{2^n} (1-\alpha)$$
(where $n$ is the number of qubits).

In [ ]:
#Create an RBFitter object and fit the data
rbfit = rb.fitters.RBFitter(result_list, xdata)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
rbfit.plot_rb_data(ax=ax)